# Data Exploration & Alpha Factor Demo

Two sections:
1. **Database sanity check** — verify `stock_data.db` is built correctly and preview sample data (OHLCV + amount, adj_factor, industry, missing values).
2. **Alpha computation** — compute Alpha#6, #12, #38, #41, #101 via `Alpha101`.

> **Prerequisites**: run `engine.download_data()` first so that `data/stock_data.db` exists.

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
from data_loader import DataEngine
from alphas import Alpha101

In [ ]:
engine = DataEngine()
data = engine.load_data()

df_price    = data['df_price']
df_mv       = data['df_mv']
df_industry = data['df_industry']
df_adj      = data['df_adj']

print('Tables loaded.')
print(f'  daily_price : {df_price.shape}  (rows = date×code combinations, cols include amount)')
print(f'  df_mv       : {df_mv.shape}')
print(f'  stock_info  : {df_industry.shape}')
print(f'  adj_factor  : {df_adj.shape}')

---
## Part 1 — Database Sanity Check

### 1.1 Daily Price (OHLCV + Amount)

In [ ]:
dates = df_price.index.get_level_values('date')
codes_idx = df_price.index.get_level_values('code')
print(f'Date range   : {dates.min()}  ->  {dates.max()}')
print(f'Unique stocks: {codes_idx.nunique()}')
print(f'Columns      : {df_price.columns.tolist()}')
df_price.head(10)

### 1.2 Adjustment Factor (adj_factor)

In [ ]:
# adj_factor overview
print(f'adj_factor shape: {df_adj.shape}')
print(f'Date range: {df_adj.index.get_level_values("date").min()}  ->  {df_adj.index.get_level_values("date").max()}')
print(f'Unique stocks: {df_adj.index.get_level_values("code").nunique()}')
df_adj.head(10)

In [ ]:
# Show adj_factor history for one sample stock (pivot to wide form: dates × codes)
sample_codes = df_adj.index.get_level_values('code').unique()[:4].tolist()
adj_wide = df_adj['adj_factor'].unstack('code')[sample_codes]
print('adj_factor (wide form) — last 5 trading dates:')
adj_wide.tail(5)

### 1.3 Market Cap (total_mv)

In [ ]:
df_mv.head(10)

### 1.4 Industry Distribution

In [ ]:
df_industry.head(10)

In [ ]:
df_industry['industry'].value_counts()

### 1.5 Missing Data Check

In [ ]:
print('=== daily_price null counts ===')
print(df_price.isnull().sum())
print()
print('=== df_mv null counts ===')
print(df_mv.isnull().sum())
print()
print('=== adj_factor null counts ===')
print(df_adj.isnull().sum())
print()
# stocks present in daily_price but missing from adj_factor
price_codes = set(df_price.index.get_level_values('code').unique())
adj_codes   = set(df_adj.index.get_level_values('code').unique())
missing_adj = price_codes - adj_codes
print(f'Stocks in daily_price but missing adj_factor: {len(missing_adj)}')

---
## Part 2 — Alpha Factor Computation

Using `Alpha101` from `src/alphas.py`, which implements alphas from  
*'101 Formulaic Alphas'* (Kakushadze, 2015).

**Price adjustment** (`adj_type` parameter):
- `'forward'` (default): `P × adj_factor / adj_factor_latest` — prices anchored to the latest date in the range
- `'backward'`          : `P × adj_factor` — prices reflect full historical appreciation
- `'raw'`               : no adjustment applied

**VWAP**: `amount × 10 / vol` (精确值); falls back to `(H+L+C)/3` when unavailable.  
`vol` and `amount` are never adjusted regardless of `adj_type`.

Raw factor values (NaN, inf) are preserved — data cleaning comes later.

In [ ]:
# Default: forward-adjusted prices + precise VWAP
# Change adj_type to 'backward' or 'raw' to switch mode
adj_type = 'forward'
alpha = Alpha101(data, adj_type=adj_type)

print(f'Alpha101 initialized  (adj_type={adj_type!r}).')
print(f'  Price matrix shape (dates × codes): {alpha.close.shape}')

# Verify vwap is not simply (H+L+C)/3 — spot-check one cell
sample_date = alpha.close.index[10]
sample_code = alpha.close.columns[0]
print(f'\n  VWAP spot-check  [{sample_date}, {sample_code}]:')
print(f'    alpha.vwap  = {alpha.vwap.loc[sample_date, sample_code]:.4f}')

/Users/infinity/Desktop/Career/Quant/Multi-factor Model for Stock Selection/Multi-factor-Model-for-Stock-Selection/notebooks/../src/alphas.py:136: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  self.returns  = self.close.pct_change()


### 2.1 Individual Alphas (wide form: dates × codes)

In [ ]:
# Alpha#6: -1 * correlation(open, volume, 10)
a6 = alpha.alpha006()
print('Alpha#6  shape:', a6.shape)
a6.tail(5).iloc[:, :6]   # last 5 dates, first 6 stocks

In [ ]:
# Alpha#12: sign(delta(volume, 1)) * (-1 * delta(close, 1))
a12 = alpha.alpha012()
print('Alpha#12 shape:', a12.shape)
a12.tail(5).iloc[:, :6]

In [ ]:
# Alpha#38: (-1 * rank(ts_rank(close, 10))) * rank(close / open)
a38 = alpha.alpha038()
print('Alpha#38 shape:', a38.shape)
a38.tail(5).iloc[:, :6]

In [ ]:
# Alpha#41: sqrt(high * low) - vwap
a41 = alpha.alpha041()
print('Alpha#41 shape:', a41.shape)
a41.tail(5).iloc[:, :6]

In [ ]:
# Alpha#101: (close - open) / (high - low + 0.001)
a101 = alpha.alpha101()
print('Alpha#101 shape:', a101.shape)
a101.tail(5).iloc[:, :6]

### 2.2 Combined Alpha DataFrame  (MultiIndex: date × code)

In [ ]:
df_alphas = alpha.get_all_alphas()
print('Combined alpha DataFrame shape:', df_alphas.shape)
print('Columns:', df_alphas.columns.tolist())
df_alphas.head(10)

alpha006  alpha012  alpha038  alpha041  alpha101
date     code                                                       
20240102 600028.SH       NaN       NaN       NaN -0.008627  0.439288
         600030.SH       NaN       NaN       NaN  0.097513 -0.909002
         600031.SH       NaN       NaN       NaN -0.016038 -0.212745
         600036.SH       NaN       NaN       NaN  0.073553 -0.922555
         600050.SH       NaN       NaN       NaN  0.030825 -0.638170
         600104.SH       NaN       NaN       NaN  0.017720 -0.410523
         600111.SH       NaN       NaN       NaN  0.061363 -0.554010
         600150.SH       NaN       NaN       NaN -0.072952  0.158729
         600276.SH       NaN       NaN       NaN  0.219135 -0.712300
         600309.SH       NaN       NaN       NaN  0.237141 -0.897138

### 2.3 Descriptive Statistics

In [ ]:
df_alphas.describe()

### 2.4 NaN Coverage per Alpha

In [ ]:
total = len(df_alphas)
null_pct = df_alphas.isnull().sum() / total * 100
null_pct.rename('NaN %').to_frame()

### 2.5 Cross-Sectional Snapshot on the Latest Date

In [ ]:
latest_date = df_alphas.index.get_level_values('date').max()
print(f'Latest date: {latest_date}')
df_alphas.loc[latest_date].dropna().head(10)

---
## Part 3 — Factor Cleaning (FactorCleaner)

Using `FactorCleaner` from `preprocessor.py` to convert raw alphas into model-ready factors.

Pipeline per alpha, per trading date (cross-sectional):
1. **Sanity check** — replace ±inf with NaN  
2. **Winsorize** — MAD-based clipping (Median ± 3 × 1.4826 × MAD)  
3. **Pre-standardize** — Z-score  
4. **Neutralize** — OLS residuals vs. log(market_cap) + industry dummies  
5. **Final standardize** — Z-score on residuals  
6. **Fill NaN with 0** — neutral value for stocks excluded from regression

In [ ]:
from preprocessor import FactorCleaner

cleaner = FactorCleaner(data)
df_clean_factors = cleaner.process_all(df_alphas)

print('Cleaned factor DataFrame shape:', df_clean_factors.shape)
print('Columns:', df_clean_factors.columns.tolist())
df_clean_factors.head(10)

### 3.1 Descriptive Statistics (Cleaned Factors)

In [ ]:
df_clean_factors.describe()

### 3.2 Zero-fill Rate (originally NaN after neutralization)

In [ ]:
total = len(df_clean_factors)
zero_pct = (df_clean_factors == 0.0).sum() / total * 100
zero_pct.rename('Zero-filled %').to_frame()

### 3.3 Cross-Sectional Snapshot on the Latest Date (Cleaned)

In [ ]:
latest_date = df_clean_factors.index.get_level_values('date').max()
print(f'Latest date: {latest_date}')
df_clean_factors.loc[latest_date].head(10)